In [20]:
import numpy as np
from rdkit import Chem
import pandas as pd
import csv
import copy
from tqdm import tqdm
from IPython.display import clear_output

In [21]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from rdkit import Chem
from rdkit.Chem import BRICS
from rdkit.Chem import MolToSmiles, MolFromSmiles

In [22]:
from molecules.conversion import (
    mols_from_smiles, mol_to_smiles, mols_to_smiles, canonicalize)
from molecules.fragmentation import fragment_iterative, reconstruct
from molecules.properties import add_property
from molecules.structure import (
    add_atom_counts, add_bond_counts, add_ring_counts)
from molecules.conversion import mols_from_smiles
from collections import OrderedDict

In [6]:
def _ordered_dict(lst):
    return OrderedDict(zip(lst, [0] * len(lst)))

In [7]:
def count_atoms(mol, atomlist):
    count = _ordered_dict(atomlist)
    if mol:
        for atom in mol.GetAtoms():
            symbol = atom.GetSymbol()
            if symbol not in count:
                count["Other"] += 1
            else:
                count[symbol] += 1
    return count

In [8]:
def count_bonds(mol, bondlist):
    count = _ordered_dict(bondlist)
    if mol:
        mol = copy.deepcopy(mol)
        Chem.Kekulize(mol, clearAromaticFlags=True)
        for bond in mol.GetBonds():
            count[str(bond.GetBondType())] += 1
    return count

In [9]:
def count_rings(mol, ringlist):
    ring_sizes = {i: r for (i, r) in zip(range(3, 7), ringlist)}
    count = _ordered_dict(ringlist)
    if mol:
        ring_info = Chem.GetSymmSSSR(mol)
        for ring in ring_info:
            ring_length = len(list(ring))
            if ring_length in ring_sizes:
                ring_name = ring_sizes[ring_length]
                count[ring_name] += 1
    return count

In [10]:
import pandas as pd
from joblib import Parallel, delayed

from rdkit.Chem import Crippen, QED

from molecules.conversion import mols_from_smiles
from molecules.sascorer.sascorer import calculateScore


def logp(mol):
    return Crippen.MolLogP(mol) if mol else None


def mr(mol):
    return Crippen.MolMR(mol) if mol else None


def qed(mol):
    return QED.qed(mol) if mol else None


def sas(mol):
    return calculateScore(mol) if mol else None


def add_property(dataset, name):
    fn = {"qed": qed, "SAS": sas, "logP": logp, "mr": mr}[name]
    smiles = dataset.smiles.tolist()
    mols = mols_from_smiles(smiles)
    prop = [fn(mol) for mol in mols]
    new_data = pd.DataFrame(prop, columns=[name])
    return pd.concat([dataset, new_data], axis=1, sort=False)

In [11]:
def replace_last(s, old, new):
    s_reversed = s[::-1]
    old_reversed = old[::-1]
    new_reversed = new[::-1]

    # Replace the first occurrence in the reversed string
    s_reversed = s_reversed.replace(old_reversed, new_reversed, 1)

    # Reverse the string back to original order
    return s_reversed[::-1]

def check_reconstruction(frags, frag_1, frag_2, orig_smi):
    try:
        #print("Reconstructing...")
        frags_test = frags.copy()
        frags_test.append(frag_1)
        frags_test.append(frag_2)
        frag_2_re = frags_test[-1]
        for i in range(len(frags_test)-1):
            frag_1_re = frags_test[-1*i-2]
            recomb = replace_last(frag_2_re, "*", frag_1_re.replace("*", "",1))
            recomb_canon = MolToSmiles(MolFromSmiles(Chem.CanonSmiles(recomb)),rootedAtAtom = 1)
            frag_2_re = recomb_canon
        orig_smi_canon = MolToSmiles(MolFromSmiles(Chem.CanonSmiles(orig_smi)),rootedAtAtom = 1)
        if recomb_canon == orig_smi_canon:
            #print("Reconstruction successful")
            #print("Original Smiles:", orig_smi, "Fragment 1:" , frag_1, "Fragment 2: ", frag_2, "Reconstruction: ", recomb_canon)
            return True
        else:
            #print("Reconstruction failed")
            #print("True Smiles:", smi, "Fragment 1:" , frag_1, "Fragment 2: ", frag_2, "Reconstruction: ", recomb_canon)
            return False
    except:
        #print("Reconstruction failed")
        #print("True Smiles:", smi, "Fragment 1:" , frag_1, "Fragment 2: ", frag_2, "Reconstruction: ", recomb_canon)
        return False

def fragment_recursive(mol_smi_orig, mol_smi, frags, counter, frag_list_len):
    fragComplete = False
    try:
        counter += 1
        mol = MolFromSmiles(mol_smi)
        bonds = list(BRICS.FindBRICSBonds(mol))
        if len(bonds) <= frag_list_len:
            frags.append(MolToSmiles(MolFromSmiles(Chem.CanonSmiles(mol_smi)), rootedAtAtom=1))
            #rint("Final Fragment: ", mol_smi, "Number of BRIC bonds: ", len(bonds))
            fragComplete = True
            return fragComplete

        idxs, labs = list(zip(*bonds))

        bond_idxs = []
        for a1, a2 in idxs:
            bond = mol.GetBondBetweenAtoms(a1, a2)
            bond_idxs.append(bond.GetIdx())

        order = np.argsort(bond_idxs).tolist()
        bond_idxs = [bond_idxs[i] for i in order]
        for bond in bond_idxs:
            broken = Chem.FragmentOnBonds(mol,
                                        bondIndices=[bond],
                                        dummyLabels=[(0, 0)])
            head, tail = Chem.GetMolFrags(broken, asMols=True)
            head_bric_bond_no = len(list(BRICS.FindBRICSBonds(head)))
            tail_bric_bond_no = len(list(BRICS.FindBRICSBonds(tail)))
            if head_bric_bond_no <= frag_list_len:
                head_smi = Chem.CanonSmiles(MolToSmiles(head))
                tail_smi = MolToSmiles(MolFromSmiles(Chem.CanonSmiles(MolToSmiles(tail))), rootedAtAtom=1)
                if check_reconstruction(frags, head_smi, tail_smi, mol_smi_orig):
                    frags.append(head_smi)
                    #print("Recursed: ", mol_smi, "Bond: ", bond, "Terminal: ", head_smi, "Number of BRIC bonds: ", head_bric_bond_no, "Recurse: ", tail_smi)
                    fragComplete = fragment_recursive(mol_smi_orig, tail_smi, frags, counter, frag_list_len = 0)  
                    if fragComplete:
                        return frags
                elif len(bond_idxs) == 1:
                    frags.append(MolToSmiles(MolFromSmiles(Chem.CanonSmiles(mol_smi)), rootedAtAtom=1))
                    #print("Final Fragment: ", mol_smi, "Number of BRIC bonds: ", len(bonds))
                    fragComplete = True
                    return frags
                elif bond == bond_idxs[-1]:
                    fragComplete = fragment_recursive(mol_smi_orig, MolToSmiles(MolFromSmiles(Chem.CanonSmiles(mol_smi)), rootedAtAtom=1), frags, counter, frag_list_len + 1)
                    if fragComplete:
                        return frags
            elif tail_bric_bond_no <= frag_list_len:
                tail_smi = Chem.CanonSmiles(MolToSmiles(tail))
                head_smi = MolToSmiles(MolFromSmiles(Chem.CanonSmiles(MolToSmiles(head))), rootedAtAtom=1)
                if check_reconstruction(frags, tail_smi, head_smi, mol_smi_orig):
                    frags.append(tail_smi)
                    #print("Recursed: ", mol_smi, "Bond: ", bond,  "Terminal: ", tail_smi, "Number of BRIC bonds: ", tail_bric_bond_no, "Recurse: ", head_smi)
                    fragComplete = fragment_recursive(mol_smi_orig, head_smi, frags, counter, frag_list_len = 0)  
                    if fragComplete:
                        return frags
                elif len(bond_idxs) == 1:
                    frags.append(MolToSmiles(MolFromSmiles(Chem.CanonSmiles(mol_smi)), rootedAtAtom=1))
                    #print("Final Fragment: ", mol_smi, "Number of BRIC bonds: ", len(bonds))
                    fragComplete = True
                    return frags
                elif bond == bond_idxs[-1]:
                    fragComplete = fragment_recursive(mol_smi_orig, MolToSmiles(MolFromSmiles(Chem.CanonSmiles(mol_smi)), rootedAtAtom=1), frags, counter, frag_list_len + 1)
                    if fragComplete:
                        return frags
    except Exception:
        pass

In [12]:
def break_into_fragments(mol, smi):
    #frags = fragment_iterative(mol)
    frags = []
    fragment_recursive(smi, smi, frags, 0, 0)

    if len(frags) == 0:
        return smi, np.nan, 0

    if len(frags) == 1:
        return smi, smi, 1
    """
    rec, frags = reconstruct(frags)
    if rec and mol_to_smiles(rec) == smi:
        fragments = mols_to_smiles(frags)
        return smi, " ".join(fragments), len(frags)

    return smi, np.nan, 0
    """
    clear_output()
    return smi, " ".join(frags), len(frags)
    

In [13]:
def add_fragments(dataset):
    smiles = dataset.smiles.tolist()
    mols = mols_from_smiles(smiles)
    results = [break_into_fragments(m, s) for m, s in zip(mols, smiles)]
    smiles, fragments, lengths = zip(*results)
    dataset["smiles"] = smiles
    dataset["fragments"] = fragments
    dataset["n_fragments"] = lengths
    
    return dataset

# Complete Pipeline

In [14]:
#dataset = pd.read_csv('\home\teddy_t\UCL\drug_discovery\models\Datasets\guacamol_v1_test.smiles')
data_type = 'train'
#dataset = pd.read_csv('G:/My Drive/Consultancy/UCL/Research Assistant/Datasets/guacamol_v1_'+data_type+'.smiles')[100000:400050]
dataset = pd.read_csv('G:/My Drive/Consultancy/UCL/Research Assistant/fragment-vae/DATA/General/gaucamol_'+data_type+'_300002').head(1000)
dataset = dataset[['smiles']]

In [15]:
ind_arr = []
dataset.size
i = 0
while i < dataset.size:
    ind_arr.append(i)
    i += 100001
ind_arr.append(dataset.size)

In [16]:
ind_arr

[0, 1000]

In [17]:
atomlist = [
        "C",
        "F",
        "N",
        "O",
        "Other"]
bondlist =  [
        "SINGLE",
        "DOUBLE",
        "TRIPLE"
    ]
ringlist = [
        "Tri",
        "Quad",
        "Pent",
        "Hex"
    ]
properties = [
    "qed",
    "logP",
    "SAS",
    "mr"
]

In [18]:
for i in tqdm(range(0, len(ind_arr)-1)):
    ind_start = ind_arr[i]
    ind_end = ind_arr[i+1]-1
    df = dataset.iloc[ind_start:ind_end,:]
    df = df.drop_duplicates()
    df = df[~df.smiles.str.contains("\.")]
    smiles = df.smiles.tolist()
    print("Canonicalizing ...")
    df.smiles = [canonicalize(smi, clear_stereo=True) for smi in smiles]
    df = df[df.smiles.notnull()].reset_index(drop=True)
    smiles = df.smiles.tolist()
    mols = mols_from_smiles(smiles)
    #add atom count
    print("Adding atom count ...")
    counts = [count_atoms(mol, atomlist) for mol in mols]
    df = pd.concat([df, pd.DataFrame(counts)], axis=1, sort=False)
    #add bond count
    print("Adding bond count ...")
    counts = [count_bonds(mol, bondlist) for mol in mols]
    df = pd.concat([df, pd.DataFrame(counts)], axis=1, sort=False)
    #add ring count
    print("Adding ring count ...")
    counts = [count_rings(mol, ringlist) for mol in mols]
    df = pd.concat([df, pd.DataFrame(counts)], axis=1, sort=False)
    #add properties
    print("Adding properties ...")
    for prop in properties:
        if prop not in dataset.columns:
            df = add_property(df, prop)
    #add fragments
    print("Adding fragments ...")
    #smiles = df.smiles.tolist()
    #mols = mols_from_smiles(smiles)
    #smiles, fragments, lengths = zip(*results)
    df = add_fragments(df)
    #finalise and save dataset
    df = df[["smiles","fragments","n_fragments","C","F","N","O","Other","SINGLE","DOUBLE","TRIPLE","Tri","Quad","Pent","Hex","logP","mr","qed","SAS"]]
    df.to_csv("D:/UCL/Dissertation Drug Discovery/Datasets/CHEMBLE/gaucamol_"+data_type+"_"+"300002"+"_recurse", index=False)
     
    



100%|██████████| 1/1 [01:18<00:00, 78.80s/it]


In [41]:
index = np.argmax(df.n_fragments)
print(df.iloc[index].n_fragments)
print(df.iloc[index].fragments)
print(df.iloc[index].smiles)

23
*S(=O)(=O)c1cccc(C2CN(C)Cc3c(Cl)cc(Cl)cc32)c1 *N* *CC* *O* *CC* *O* *CC* *O* *CC* *O* *CC* *O* *CC* *O* *CC* *O* *CC* *O* *CC* *O* *CC* *O* C(*)CN=[N+]=[N-]
CN1Cc2c(Cl)cc(Cl)cc2C(c2cccc(S(=O)(=O)NCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCN=[N+]=[N-])c2)C1


In [23]:
# File Paths
rel_path = "G:/My Drive/Consultancy/UCL/Research Assistant/fragment-vae/DATA/ZINC"
file_paths = [rel_path + "/zinc_processed_recurse_with_stereo100000",
                rel_path + "/zinc_processed_recurse_with_stereo200001",
                rel_path + "/zinc_processed_recurse_with_stereo249454"
                ]
# Create a list to store DataFrames
dfs = []

# Loop through the file paths and read each file's data into a DataFrame
for file in file_paths:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)


In [24]:
#remove rows with 1 fragment
combined_df_frags_only = combined_df[combined_df.n_fragments != 1].reset_index(drop=True)
combined_df_frags_only.head()
combined_df_frags_only.to_csv("G:/My Drive/Consultancy/UCL/Research Assistant/fragment-vae/DATA/ZINC/PROCESSED/ZINC_processed_recurse_frags_only_w_stereo", index=True)

In [25]:
# Save the combined DataFrame to a new file
combined_df.to_csv("G:/My Drive/Consultancy/UCL/Research Assistant/fragment-vae/DATA/ZINC/PROCESSED/train_full_recomb_recurse_w_stereo", index=True)

In [26]:
combined_df.head()

,smiles,fragments,n_fragments,C,F,N,O,Other,SINGLE,DOUBLE,TRIPLE,Tri,Quad,Pent,Hex,logP,mr,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1,*C(C)(C)C *C(=O)Cc1coc2ccc(*)cc12 *N* *c1ccccc1F,4,20,1,1,2,0,18,8,0,0,0,1,2,5.05060,93.6477,0.731901,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1,*C1C[C@@H](C)C[C@H](C)C1 *N* *c1cncc(-c2nncn2C)c1,3,16,0,5,0,0,18,5,0,0,0,1,2,3.11370,83.8237,0.941112,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,*N1CCCC1 *c1ccccc1 *O[C@H](*)C(*)=O *c1ccc(-c2...,4,25,0,2,2,0,21,10,1,0,0,1,3,4.96778,112.2180,0.626105,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,*CC *O* *C(*)=O *[C@@H]1CCCN(C(=O)c2nc(-c3ccc(...,4,24,0,3,3,0,26,7,0,0,0,1,2,4.00022,115.2155,0.716225,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,*SC1=C(C#N)C2(CCCCC2)[C@@H](C#N)C([O-])=N1 *c1...,3,20,0,4,2,2,22,6,2,0,0,0,3,3.60956,107.2347,0.809572,4.035182


In [40]:
pd.read_csv('G:/My Drive/Consultancy/UCL/Research Assistant/Datasets/guacamol_v1_'+data_type+'.smiles')[99999:100001]

,smiles
99999,N#Cc1cc2c(cc1Cl)Sc1nccn1S2(=O)=O
100000,CCN(CC)CC(=O)Nc1nc(-c2ccc(F)cc2Cl)cs1
